In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv('../application/.env'))

True

In [2]:
from langchain.indexes import SQLRecordManager, index
import os
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores.pgvector import PGVector


host = os.getenv("PG_VECTOR_HOST")
user = os.getenv("PG_VECTOR_USER")
password = os.getenv("PG_VECTOR_PASSWORD")
COLLECTION_NAME = os.getenv("PGDATABASE")
CONNECTION_STRING = f"postgresql+psycopg2://{user}:{password}@{host}:5432/{COLLECTION_NAME}"

namespace = f"pgvector/{COLLECTION_NAME}"
record_manager = SQLRecordManager(
    namespace, db_url=CONNECTION_STRING
)

embeddings = OpenAIEmbeddings()

vector_store = PGVector(
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

c:\Users\reesh\anaconda3\envs\course\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainPendingDeprecationWarning: This class is pending deprecation and may be removed in a future version. You can swap to using the `PGVector` implementation in `langchain_postgres`. Please read the guidelines in the doc-string of this class to follow prior to migrating as there are some differences between the implementations. See https://github.com/langchain-ai/langchain-postgres for details aboutthe new implementation.
  warn_deprecated(
c:\Users\reesh\anaconda3\envs\course\Lib\site-packages\langchain_community\vectorstores\pgvector.py:322: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing

In [3]:
index([], record_manager, vector_store, cleanup="full", source_id_key="source")

{'num_added': 0, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [11]:
import os
from langchain_community.document_loaders import AzureBlobStorageContainerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_documents(chunk_size=200, chunk_overlap=20):
    conn_str = os.getenv("BLOB_CONN_STRING")
    container_name = os.getenv("BLOB_CONTAINER")

    if conn_str is None or container_name is None:
        raise ValueError("Environment variables for BLOB_CONN_STRING or BLOB_CONTAINER are not set.")

    loader = AzureBlobStorageContainerLoader(conn_str=conn_str, container=container_name)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len, is_separator_regex=False)
    return text_splitter.split_documents(data)


In [12]:
docs = load_and_split_documents()
docs

[Document(page_content='At La Tavola Calda, we believe in the art of simplicity and the beauty of tradition. Our carefully curated pizza menu, featuring just four exquisite varieties, is a testament to our commitment to', metadata={'source': 'C:\\Users\\reesh\\AppData\\Local\\Temp\\tmpzym_t2id/udemycontainerazure/food.txt'}),
 Document(page_content='our commitment to quality over quantity. Each pizza represents a distinct aspect of Italian culinary heritage, crafted with the freshest ingredients and the utmost care.', metadata={'source': 'C:\\Users\\reesh\\AppData\\Local\\Temp\\tmpzym_t2id/udemycontainerazure/food.txt'}),
 Document(page_content='We offer four pizzas: Margherita, Salami, Quattro Formaggi, Marshmallow', metadata={'source': 'C:\\Users\\reesh\\AppData\\Local\\Temp\\tmpzym_t2id/udemycontainerazure/food.txt'}),
 Document(page_content="Pizza Margherita - The Classic Perfection: Paying homage to the colors of the Italian flag, our Margherita is a blend of ripe tomatoes, fresh 

In [13]:
index(docs, record_manager, vector_store, cleanup="full", source_id_key="source")

{'num_added': 24, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 24}

In [14]:
docs = load_and_split_documents()
docs

[Document(page_content='At La Tavola Calda, we believe in the art of simplicity and the beauty of tradition. Our carefully curated pizza menu, featuring just four exquisite varieties, is a testament to our commitment to', metadata={'source': 'C:\\Users\\reesh\\AppData\\Local\\Temp\\tmpop454n24/udemycontainerazure/food.txt'}),
 Document(page_content='our commitment to quality over quantity. Each pizza represents a distinct aspect of Italian culinary heritage, crafted with the freshest ingredients and the utmost care.', metadata={'source': 'C:\\Users\\reesh\\AppData\\Local\\Temp\\tmpop454n24/udemycontainerazure/food.txt'}),
 Document(page_content='We offer four pizzas: Margherita, Salami, Quattro Formaggi, Marshmallow', metadata={'source': 'C:\\Users\\reesh\\AppData\\Local\\Temp\\tmpop454n24/udemycontainerazure/food.txt'}),
 Document(page_content="Pizza Margherita - The Classic Perfection: Paying homage to the colors of the Italian flag, our Margherita is a blend of ripe tomatoes, fresh 

In [15]:
index(docs, record_manager, vector_store, cleanup="full", source_id_key="source")

{'num_added': 24, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 24}

In [16]:
import os
from langchain_community.document_loaders import AzureBlobStorageContainerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_documents(chunk_size=200, chunk_overlap=20):
    conn_str = os.getenv("BLOB_CONN_STRING")
    container_name = os.getenv("BLOB_CONTAINER")

    if conn_str is None or container_name is None:
        raise ValueError("Environment variables for BLOB_CONN_STRING or BLOB_CONTAINER are not set.")

    loader = AzureBlobStorageContainerLoader(conn_str=conn_str, container=container_name)
    data = loader.load()
    for doc in data:
        doc.metadata["source"] = os.path.basename(doc.metadata["source"])

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len, is_separator_regex=False)
    return text_splitter.split_documents(data)


In [17]:
docs = load_and_split_documents()
docs

[Document(page_content='At La Tavola Calda, we believe in the art of simplicity and the beauty of tradition. Our carefully curated pizza menu, featuring just four exquisite varieties, is a testament to our commitment to', metadata={'source': 'food.txt'}),
 Document(page_content='our commitment to quality over quantity. Each pizza represents a distinct aspect of Italian culinary heritage, crafted with the freshest ingredients and the utmost care.', metadata={'source': 'food.txt'}),
 Document(page_content='We offer four pizzas: Margherita, Salami, Quattro Formaggi, Marshmallow', metadata={'source': 'food.txt'}),
 Document(page_content="Pizza Margherita - The Classic Perfection: Paying homage to the colors of the Italian flag, our Margherita is a blend of ripe tomatoes, fresh mozzarella, and basil. It's a timeless classic that lets", metadata={'source': 'food.txt'}),
 Document(page_content='classic that lets the purity and freshness of the ingredients shine through.', metadata={'source': 

In [18]:
index(docs, record_manager, vector_store, cleanup="full", source_id_key="source")

{'num_added': 24, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 24}

Lets now update something in the raw data and upload it again

In [19]:
import os
from azure.storage.blob import BlobServiceClient

folder_path = "./restaurant"

conn_str=os.getenv("BLOB_CONN_STRING")
container_name = os.getenv("BLOB_CONTAINER")


blob_service_client = BlobServiceClient.from_connection_string(conn_str=conn_str)

for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)):
        file_path = os.path.join(folder_path, filename)

        blob_client = blob_service_client.get_blob_client(container=container_name, blob=filename)

        with open(file_path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)

In [20]:
docs = load_and_split_documents()
for doc in docs[0:3]:
    print(doc)

page_content='At La Tavola Calda, we believe in the art of simplicity and the beauty of tradition. Our carefully curated pizza menu, featuring just four exquisite varieties, is a testament to our commitment to' metadata={'source': 'food.txt'}
page_content='our commitment to quality over quantity. Each pizza represents a distinct aspect of Italian culinary heritage, crafted with the freshest ingredients and the utmost care.' metadata={'source': 'food.txt'}
page_content='We offer four pizzas: Margherita, Salami, Quattro Formaggi, Marshmallow' metadata={'source': 'food.txt'}


In [21]:
index(docs, record_manager, vector_store, cleanup="full", source_id_key="source")

{'num_added': 2, 'num_updated': 0, 'num_skipped': 19, 'num_deleted': 5}

We now have linked Blob Storage And PgVector and keep the raw docs and the indexed documents in sync. But in a real app we want to handle that automatically. Thats was Azure functions can do for us. We will take a look at Azure Functions later in the course